In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/icr-identify-age-related-conditions/sample_submission.csv
/kaggle/input/icr-identify-age-related-conditions/greeks.csv
/kaggle/input/icr-identify-age-related-conditions/train.csv
/kaggle/input/icr-identify-age-related-conditions/test.csv


In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
import os

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
train_data = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/train.csv')
test_data = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/test.csv')

In [4]:
train_data = train_data[train_data['BQ'].notna()]

train_data['EL'].fillna(train_data['EL'].mean(), inplace=True)
train_data['CC'].fillna(train_data['CC'].mean(), inplace=True)
train_data.CB.fillna(train_data.CB.median(), inplace=True)
train_data.FS.fillna(train_data.FS.median(), inplace=True)
train_data.FC.fillna(train_data.FC.median(), inplace=True)

train_data.reset_index(inplace=True)
train_data.drop('index', axis=1, inplace=True)

In [5]:
X = train_data.drop(['Id', 'Class'], axis=1)
y = train_data.drop(X.columns, axis=1).drop(['Id'], axis=1)
y_class0 = pd.DataFrame(np.logical_xor(y['Class'].values, 1).astype(int), columns=["Class1"], index=y.index)
y = pd.concat([y, y_class0], axis=1)

In [6]:
one_hot_encoder = OneHotEncoder(drop='first', sparse=False)
one_hot_encoder.fit(X['EJ'].unique().reshape(-1,1))
X['EJ'] = pd.DataFrame(one_hot_encoder.transform(X['EJ'].to_numpy().reshape(-1,1)))

/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(445, 56) (112, 56) (445, 2) (112, 2)


In [8]:
class WideAndDeepModel(tf.keras.Model):
    def __init__(self, units=256, activation='relu', **kwargs):
        super().__init__(**kwargs)
        rate = 0.3
        self.norm_layer_wide = tf.keras.layers.Normalization()
        self.norm_layer_deep = tf.keras.layers.Normalization()
        self.dropout1 = tf.keras.layers.Dropout(rate=rate)
        self.dropout2 = tf.keras.layers.Dropout(rate=rate)
        self.hidden1 = tf.keras.layers.Dense(units, activation=activation, 
                                             kernel_initializer='he_normal', 
                                             )
        self.dropout3 = tf.keras.layers.Dropout(rate=rate)
        self.hidden2 = tf.keras.layers.Dense(units, activation=activation, 
                                             kernel_initializer='he_normal', 
                                             )
        self.dropout4 = tf.keras.layers.Dropout(rate=rate)
        # self.hidden3 = tf.keras.layers.Dense(units, activation=activation, 
        #                                      kernel_initializer='he_normal', 
        #                                      )
        # self.dropout5 = tf.keras.layers.Dropout(rate=0.2)
        self.output_ = tf.keras.layers.Dense(2, kernel_initializer='he_normal', 
                                             activation="sigmoid")
        
    def call(self, inputs):
        input_ = inputs
        # rate=0.2
        norm_wide = self.norm_layer_wide(input_)
        norm_deep = self.norm_layer_deep(input_)
        dropout1 = self.dropout1(norm_wide)
        dropout2 = self.dropout2(norm_deep)
        hidden1 = self.hidden1(dropout2)
        dropout3 = self.dropout3(hidden1)
        hidden2 = self.hidden2(dropout3)
        dropout4 = self.dropout4(hidden2)
        # hidden3 = self.hidden2(dropout4)
        # dropout5 = self.dropout4(hidden3)
        concat = tf.keras.layers.concatenate([dropout1, dropout4])
        output = self.output_(concat)
        return output

In [9]:
model = WideAndDeepModel(units=256, activation="relu")
model.call(X_train)
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(loss = 'binary_crossentropy', optimizer=optimizer, metrics=["accuracy"])
model.norm_layer_wide.adapt(X_train)
model.norm_layer_deep.adapt(X_train)

In [10]:
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint('Checkpoint', save_weights_only=True)
earlystopping_cb = tf.keras.callbacks.EarlyStopping(patience=100, restore_best_weights=True)
# tfboard_cb = tf.keras.callbacks.TensorBoard(run_logdir)

# callbacks = [checkpoint_cb, earlystopping_cb, tfboard_cb]
callbacks = [checkpoint_cb, earlystopping_cb]

In [11]:
# tf.keras.backend.set_value(model.optimizer.learning_rate, 1e-3)
model.fit(X_train, y_train, epochs=2050, initial_epoch=0, batch_size=16, validation_split=0.1, callbacks=callbacks)

Epoch 1/2050
25/25 [==============================] - 1s 15ms/step - loss: 0.7185 - accuracy: 0.7675 - val_loss: 0.3683 - val_accuracy: 0.8667
Epoch 2/2050
25/25 [==============================] - 0s 5ms/step - loss: 0.4510 - accuracy: 0.8700 - val_loss: 0.3055 - val_accuracy: 0.8667
Epoch 3/2050
25/25 [==============================] - 0s 5ms/step - loss: 0.3631 - accuracy: 0.8625 - val_loss: 0.2880 - val_accuracy: 0.8889
Epoch 4/2050
25/25 [==============================] - 0s 5ms/step - loss: 0.3545 - accuracy: 0.8725 - val_loss: 0.3277 - val_accuracy: 0.8667
Epoch 5/2050
25/25 [==============================] - 0s 5ms/step - loss: 0.3070 - accuracy: 0.8825 - val_loss: 0.3064 - val_accuracy: 0.8889
Epoch 6/2050
25/25 [==============================] - 0s 5ms/step - loss: 0.2959 - accuracy: 0.8850 - val_loss: 0.2865 - val_accuracy: 0.8889
Epoch 7/2050
25/25 [==============================] - 0s 5ms/step - loss: 0.2298 - accuracy: 0.9175 - val_loss: 0.3349 - val_accuracy: 0.8889
Epoch

In [12]:
model.evaluate(X_test, y_test)

4/4 [==============================] - 0s 2ms/step - loss: 0.4179 - accuracy: 0.9196


[0.4179363250732422, 0.9196428656578064]

In [13]:
test_data = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/test.csv')
X1 = test_data
X1['EJ'] = pd.DataFrame(one_hot_encoder.transform(X1['EJ'].to_numpy().reshape(-1,1)))
columns = X.columns

In [14]:
for column in columns:
    X1[column].fillna(X1[column].median(), inplace=True)

In [15]:
pd.concat([pd.DataFrame(X1['Id']), pd.DataFrame(model.predict(X1.drop('Id', axis=1)), columns=['class_0', 'class_1'])], axis=1).to_csv('submission.csv', index=False)

1/1 [==============================] - 0s 100ms/step
